# Import Figure8 labels to Custom Vision Service

In [ ]:
# install packages if needed
import sys
!{sys.executable} -m pip install azure-cognitiveservices-vision-customvision
!{sys.executable} -m pip install azureml-sdk
!{sys.executable} -m pip install azureml-contrib-dataset
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install Pillow

In [ ]:
import json, os, shutil, requests, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from io import BytesIO
from numpy import asarray

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
# azureml-contrib-dataset of version 1.0.72 or higher is required

## 1. Set up Custom Vision project

Enter the details for your Custom Vision endpoint and training key below:

In [ ]:
ENDPOINT = 'https://YOUR_REGION.api.cognitive.microsoft.com'
training_key = "<CUSTOM VISION TRAINING KEY>"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint=ENDPOINT, credentials=credentials)

### Option 1: Create new project

The below cell will create a new Custom Vision project.  Enter a name for your project below:

Note: If you have an existing project, skip to the next cell.

In [ ]:
project_name = "<PROJECT NAME>"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print("Creating new project...")
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)
print(project.name, "project created")

### Option 2: Update existing Custom Vision project

The below cell will get an existing project by ID.  Enter your project ID below (you can retrieve this ID from your project in the [Custom Vision portal](http://customvision.ai)):

In [ ]:
project_id = "<PROJECT ID>"

# Get existing project
project = trainer.get_project(project_id = project_id) 
print(project.name, "project retrieved")

## 2. Read labeled dataset from csv

In [ ]:
filename = "<PATH TO CSV FILE WITH LABELS>" 

df = pd.read_csv(filename)
df = df[['annotation','image_url','image_name']]
df.head()

### Prep images and format tags for Custom Vision

In [ ]:
def get_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    width = img.size[0]
    height = img.size[1]
    img_bytes = image_to_bytes(img)
    return img_bytes, width, height

def get_bbox(label, width, height):
    l = label['x']/width
    t = label['y']/height
    w = label['w']/width
    h = label['h']/height  
    return l,t,w,h

def image_to_bytes(image):
    # Convert image to byte stream
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr

In [ ]:
tagged_ims = []
tags = trainer.get_tags(project.id)

num_labels = df.shape[0]

previous_url = ''
regions = []
for i, row in df.iterrows():
    
    # handle first as edge case to get pointer to previous image url
    if i == 0:
        previous_url = row['image_url']
        img_bytes, width, height = get_image_from_url(previous_url)
        filename = row['image_name']
        print(f"Processing image: {filename} - Label: {i}/{num_labels}")
        
    # get image url and label
    img_url = row['image_url']
    label = json.loads(row['annotation'])
    
    # retrieve tag object by label name 
    label_name = 'vehicle'  # TODO: read other types of labels dynamically 
    try:
        index = [x.name for x in tags].index(label_name)
        tag = tags[index]
    # create tag if it does not exist yet
    except:
        print("Creating new tag for:", label_name)
        tag = trainer.create_tag(project.id, label_name)
        tags = trainer.get_tags(project.id)
      
    # create bounding box regions
    l,t,w,h = get_bbox(label[0]['coordinates'], width, height)
    
    # when we get to a new image
    if img_url != previous_url:
        
        # create image file entry for all data from previous image and append to image set
        tagged_ims.append(ImageFileCreateEntry(name=filename, contents=img_bytes, regions=regions))
    
        regions = []
        regions.append(Region(tag_id=tag.id,left=l,top=t,width=w,height=h))
        
        # get info from new image
        img_bytes, width, height = get_image_from_url(img_url)
        filename = row['image_name']

        print(f"Processing image: {filename} - Label: {i}/{num_labels}")
        
    else:
        regions.append(Region(tag_id=tag.id,left=l,top=t,width=w,height=h))

    # update previous url to current
    previous_url = img_url

## 3. Upload images and tags to Custom Vision

In [ ]:
counter = 0

for i in range(0, len(tagged_ims), 64): 
    batch = tagged_ims[i:i+64]
    result = trainer.create_images_from_files(project.id, images=batch)

    counter += 1
    print(f"Uploaded batch {counter} of {math.ceil(len(tagged_ims)/64)}: {result.is_batch_successful}")
    
    if not result.is_batch_successful:
        for j, image in enumerate(result.images):
            print(f" Image {j} status: {image.status}")